### starting with kaggle dataset - take the app ids
### view the dataset

#### ---- set D3 - Productivity, Shopping, Finance, Social, Personalization

In [1]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('../Google-Playstore.csv')
df.head(2)

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35


In [3]:
df.shape

(2312944, 24)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       float64
 5   Installs           object 
 6   Minimum Installs   float64
 7   Maximum Installs   int64  
 8   Free               bool   
 9   Price              float64
 10  Currency           object 
 11  Size               object 
 12  Minimum Android    object 
 13  Developer Id       object 
 14  Developer Website  object 
 15  Developer Email    object 
 16  Released           object 
 17  Last Updated       object 
 18  Content Rating     object 
 19  Privacy Policy     object 
 20  Ad Supported       bool   
 21  In App Purchases   bool   
 22  Editors Choice     bool   
 23  Scraped Time       object 
dtypes: bool(4), float64(4), int64(1), object(15)
memor

In [5]:
df['Category'].value_counts()

Education                  241090
Music & Audio              154906
Tools                      143988
Business                   143771
Entertainment              138276
Lifestyle                  118331
Books & Reference          116728
Personalization             89210
Health & Fitness            83510
Productivity                79698
Shopping                    75256
Food & Drink                73927
Travel & Local              67288
Finance                     65466
Arcade                      53792
Puzzle                      51168
Casual                      50813
Communication               48167
Sports                      47483
Social                      44734
News & Magazines            42807
Photography                 35552
Medical                     32065
Action                      27555
Maps & Navigation           26722
Simulation                  23282
Adventure                   23203
Educational                 21308
Art & Design                18539
Auto & Vehicle

In [6]:
# replacing blank spaces with '_' 
df.columns =[column.replace(" ", "_") for column in df.columns]

In [7]:
## Selecting only these categories initially

#----d3-----
## Productivity, 
## Shopping, 
## Finance, 
## Social, 
## Personalization

In [8]:
df = df[df.apply(lambda x: (x["Category"] in ["Productivity", "Shopping","Finance", "Social", "Personalization"]) , axis=1)]

In [9]:
df.shape

(354364, 24)

In [ ]:
## keep only the app_name and app_id in the different dataframe

In [10]:
df_d3 = pd.DataFrame()

In [11]:
df_d3["App_Name"] = df["App_Name"]
df_d3["App_Id"] = df["App_Id"]
df_d3["Category"] = df["Category"]
df_d3["Rating"] = df["Rating"]
df_d3["Maximum_Installs"] = df["Maximum_Installs"]
df_d3["Editors_Choice"] = df["Editors_Choice"]

In [12]:
df_d3['Description'] = ''
df_d3['Privacy_Policy'] = ''

In [13]:
df_d3.tail(3)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy
2312919,Good Night Quotes,com.appmakerz.goodnightquotes,Personalization,4.8,437,False,,
2312925,GOKarli Carrera Rennbahn und Dart Onlineshop,de.gokarli.shop,Shopping,0.0,2418,False,,
2312935,Floral Wallpaper,com.arfdev.floralwallpaper,Personalization,0.0,1302,False,,


In [14]:
df_d3.reset_index(drop=True,inplace=True)

In [15]:
df_d3.tail(3)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy
354361,Good Night Quotes,com.appmakerz.goodnightquotes,Personalization,4.8,437,False,,
354362,GOKarli Carrera Rennbahn und Dart Onlineshop,de.gokarli.shop,Shopping,0.0,2418,False,,
354363,Floral Wallpaper,com.arfdev.floralwallpaper,Personalization,0.0,1302,False,,


In [ ]:
## collect the description and privacy policy link using google-play-scraper

In [16]:
from urllib.error import HTTPError

In [17]:
!pip install google-play-scraper

In [18]:
from google_play_scraper import app

In [19]:
i = 0
err_list_d3 = pd.DataFrame()

In [20]:
for index, row in df_d3[0:354364].iterrows():      #262757 ## 50000 100000 150000 200000 250000 330214
      app_id = row.App_Id
      #print ("Trying...",index,app_id)
      try:
        result = app(app_id)
        df_d3.at[index, 'Description'] = result['description']
        df_d3.at[index, 'Privacy_Policy'] = result['privacyPolicy']
    
      except (Exception, HTTPError, TypeError) as e:
        err_list_d3.at[i,'index'] = index
        err_list_d3.at[i,'app_id'] = app_id
        err_list_d3.at[i,'category'] = row.Category
        i = i + 1
        print ("Error",index,app_id)

Error 0 com.doantiepvien.crm
Error 3 np.com.NepaliCongress
Error 22 com.bobby.pitch
Error 32 com.appix.makemoney
Error 38 com.palmer.cooltopuprecharge.reseller
Error 42 zw.ac.msu.android.sponsors
Error 47 com.iwaiterapp.papillonsalatsandwiches
Error 51 com.coimbatore_musicals
Error 57 com.live.keyboard.reindeer
Error 59 com.mapyourempire.www.myelivechat
Error 64 com.itstock.talk
Error 69 com.red.black.cool.business.theme
Error 82 com.pink.cute.couple.cartoon.theme
Error 97 de.facentis.meinlotto
Error 112 com.credea.showme
Error 121 net.flowingly.app
Error 131 com.agalves.aculturadojilo
Error 132 io.theeye.mobile.bgh
Error 138 com.souryator.christian
Error 157 jupiro.apps.networkrefresher
Error 176 com.lin.waterrip
Error 179 com.kara.ai
Error 181 com.mapplico.hobimalzemeleri
Error 182 com.andromo.dev518376.app501485
Error 186 com.andromo.dev518376.app530827
Error 190 com.gmgroup.petrepet
Error 192 com.sparklearning.vmac.chatbot
Error 193 city.travel.guide.amsterdam
Error 197 com.Kawaii_

Error 1121 com.Fortunebackoffice
Error 1127 com.khalifa007.reward
Error 1131 com.aitok.tikfollowers
Error 1135 com.trueloves.foreverm
Error 1140 pubg.pubgmobilestats.freefiremobilestats.wegameforpubg.pubgbooster.pubgcratesfree.pubgtracker.pubgf
Error 1143 proyecto.atomo.developers.jlc.luciano.atomo
Error 1145 com.bazzaio.surtilider
Error 1150 com.iwa.htday
Error 1152 com.walpapapers.PurpleWallpapers
Error 1160 com.kiriril.WallpaperOSX11StyleTheme
Error 1167 com.diamedis.a1.ixdoc2
Error 1176 wallpapers.kawaii.fonkastico.com.kawaiiwallpapers
Error 1178 com.dancestudio_pro.Academie8984
Error 1184 net.kardensoft.duwar
Error 1187 com.biika.cospl
Error 1198 com.otoreport.appmmbcreload
Error 1214 com.womcd.auroraborealis1
Error 1230 br.com.mksolutions.mksac.ftthtelecom
Error 1233 pl.digitalvirgo.christmaslvp
Error 1236 com.lock.galaxylock
Error 1237 br.com.app.gpu1951737.gpu273d7cf1a43a97d6fdc7a216520e0290
Error 1245 com.qubris.freecryptofaucet
Error 1246 ru.alphana.accounting
Error 1251 org.

Error 2258 com.narvii.amino.x4371122
Error 2277 com.alphaltd.foto_gfriend_eunha
Error 2278 oldcountrycrows.com
Error 2281 com.Boxglass.ResidentVR
Error 2283 com.uzeegar.samsung.galaxy.s9.launcher.theme
Error 2285 com.cosmicmobile.lw.winter
Error 2287 com.diagramart
Error 2291 com.cheetah.keyboard.funemoji.key
Error 2305 com.tymksoftwares.jivansevabazzar
Error 2307 pl.zeto365.mobile.app55d338757210a
Error 2316 com.natsumietsu.mitsuhaandtakibestnewweallhdd
Error 2317 com.jgdbhajan.mahamritunjaya
Error 2325 faithfullyfresh.android.app
Error 2340 com.newandromo.dev28802.app825676
Error 2352 com.wochentagebilder.WochentageSpruche
Error 2356 dk.bave.CrudeOil
Error 2361 com.fampbroward.simplenexus
Error 2379 com.ifs.banking.fiid5178
Error 2387 com.mfoundry.mb.android.mb_mb5ppu45a
Error 2397 com.tadapps.how.to.stop.eye.twitching
Error 2398 com.fi6572.godough
Error 2401 com.binfire.tasks
Error 2417 ch.coop.prontofinder
Error 2433 com.newandromo.dev1050700.app1232942
Error 2436 new.live.wallpape

Error 3356 co.freestickers.lovechat.stickers
Error 3359 one.pabbas.qrmaker
Error 3362 com.bf.app483856
Error 3366 com.ncs.free.mp3.ringtone.sound.notification.alarm.asphalt.dream.drum.bass.legends.future
Error 3369 com.santrialmanar.wpapwallpaper
Error 3371 com.iimmobile.birds.live.wallpaper.free.wallpaper.video.wallpaper
Error 3373 com.mpn.vpn
Error 3376 vjd.bbnl.gm.venkateswara
Error 3380 com.shopshoeclassifieds.app
Error 3381 com.vokob.store
Error 3387 co.com.AlHafer
Error 3395 com.phonegap.mobilefarma
Error 3407 com.chiralcode.wallpaper.galaxy
Error 3408 com.ntv.labolataimporta
Error 3412 com.parrot.parrotinphone
Error 3420 com.hd.game.wallpapers
Error 3423 com.upphittat.upphittat
Error 3434 com.lge.themepark.aurora
Error 3441 com.kingwallpaper.carswallpapers
Error 3470 lrc.qoo10.android.qstore
Error 3472 com.mehdira.nationalanthemofnetherlands
Error 3473 com.hdflags.wallpaper.denmark.images
Error 3477 appinventor.ai_dummila80.CATALOGO_DUMMI_V2
Error 3480 co.walis
Error 3483 com.bl

Error 4542 com.hitoit.shopngo
Error 4549 com.communicationcountry.app
Error 4551 com.adamrocker.android.input.simeji.global.theme.lowpoly
Error 4557 com.tamil.keyboard.app.free
Error 4562 com.techraja.statussaver
Error 4565 com.app.belidimanalis
Error 4567 com.wedo.mall
Error 4573 com.mixlabapps.inspiringsuccessquotes
Error 4586 com.rstudio.sugawallpaper
Error 4589 com.ocen.ljphotof
Error 4590 com.chaosandlight.com
Error 4596 com.cmapps.fireskullwallpaper
Error 4598 com.triologic.jewelflowpro.jevarornaments
Error 4607 store.chekaprice.PCS
Error 4611 com.matchy.matchySets
Error 4620 com.fts.alva
Error 4627 com.SoftCode.mPay
Error 4629 nickagas.com.countdownnotificationapp
Error 4636 com.abba.kentbank
Error 4643 com.narvii.amino.x244304829
Error 4648 com.stuff.milana.eg
Error 4652 in.wpandroid.shopepie
Error 4655 com.swarm.ghnms
Error 4656 com.appsconda.clients.pmimilwaukee
Error 4657 com.ewallsznx.toywallstory
Error 4680 ringtonesfor.honor.notificationtones
Error 4682 com.GreatWallpaper

...

Error 349502 com.sairk.android.evolve.lbluefo
Error 349518 com.pet.shopie
Error 349524 com.sma.wallpaper.poppies
Error 349539 com.mc_network.app
Error 349540 com.agvistudio.goldwallpaper
Error 349541 com.youbeli.www.wellness
Error 349542 akimat_astana.ir.kz
Error 349554 com.iwaiterapp.momosdublin
Error 349555 ir.netstructures.manning
Error 349561 com.mydream786.wifiqrcode
Error 349563 com.rs.indira
Error 349566 com.topwallpaperz.windblowinginabstraction
Error 349569 com.digidrips.primeindianpharma
Error 349575 com.blue.black.wolf.theme
Error 349585 in.incarts.eyelooks
Error 349587 com.infinity.stocktarget
Error 349617 com.iwaiterapp.caesarspizza
Error 349624 com.maradecorp.marade
Error 349630 com.kajai.cars.fastcar.CoolJaguarCarHDWallpapers
Error 349636 com.emojifamily.emoji.keyboard.sticker.GingerbreadEmoji
Error 349643 com.Icewallpapers.pictures.backgrounds.photos.images.hd.free.beautiful
Error 349646 com.migrocer.maisa
Error 349657 itboost.com.mycareercolu
Error 349662 com.clujapps.

Error 350523 com.newandromo.dev6716.app68767
Error 350525 com.hamralawgroup.app
Error 350532 financeapps.bingo.livetalk.randomvideocall.livevideocall
Error 350544 com.hediyeotuzbes.mobil
Error 350554 trigyn.Mylapore
Error 350555 com.kiriril.WatchMakerLiveWallpapers
Error 350556 com.bee.keepers
Error 350562 lasing.kustom.lcr
Error 350565 com.app.p6442GE
Error 350578 in.grdtech.finalcloud
Error 350579 com.ambro.theme.keepyellowadv
Error 350580 com.ahmar.acs.baterrymeter
Error 350581 com.abfl.sbapp
Error 350584 air.org.memfcu.mb.A035
Error 350595 com.techlance.titans.multi.keyboard
Error 350600 com.fenourriandro.pitbullwallpaper
Error 350605 com.TXTSoobinwallpaperKpopHDnew.TXT_Soobin_wallpaper_Kpop_HD_new
Error 350622 com.mbianco
Error 350627 com.jalasoft.hades.wings
Error 350631 tsc.personalloansusa.com
Error 350636 com.fbt.projectTrackingDemo
Error 350641 com.sempak.jimin
Error 350646 com.wOneWorldMessenger_8155873
Error 350663 com.cccis.quickest.germania
Error 350671 com.tradenetservic

Error 351636 com.urbn
Error 351642 dhillons.co.uk
Error 351657 com.moustache.animegirlswallpaperfullhd
Error 351662 com.alpha.all.sim.network.packages
Error 351666 com.neuflora
Error 351675 boy.girl
Error 351679 it.fulvio.myfunds
Error 351681 com.alphaprods.ios.wallpaper
Error 351693 com.tmtus.tenta.pattern.lockscreenapp
Error 351695 com.gdqun.app
Error 351697 au.com.newcombeandco.app
Error 351701 com.avenhel.wolfwallpaper.animal.moon.art.wallpaper
Error 351702 com.mutiarapayment.apps
Error 351707 com.credits.diamondcredits
Error 351710 com.anaonlineshopping.clothesforwomens
Error 351714 com.com.zimax.ibileloanapp
Error 351720 com.haiie.mobile
Error 351721 com.topmostapps.shabadhazarewithhdaudio
Error 351732 com.biika.starwar
Error 351743 com.eec.prabhas
Error 351744 com.discountdeals
Error 351754 com.bhmedia.phongthuytq
Error 351764 com.hivekeepers
Error 351767 com.altothem.DachshundDogsWallpapers
Error 351780 com.shopleesboutique
Error 351787 anime.manga.wallpaper.InosukeKimetsunoYai

Error 352693 com.online.universallsurvys.best
Error 352695 com.anshul217.kong
Error 352698 com.ubi.towtow
Error 352705 com.puzzle.ks
Error 352709 com.tinkerwall.saveer.free
Error 352712 com.koloHMwallpaper.myheroacademiaboku
Error 352717 com.unilifecircles.dgdm
Error 352729 com.potenza.ciyashop_shoes
Error 352732 com.algobalance.cryptocorrelate
Error 352740 com.maheshprajapati.exoluhanwallpapers
Error 352747 com.takaful.xpert.takaful
Error 352756 com.develop.project.mobilenotefree
Error 352761 com.zuznow.ynetart
Error 352775 com.kaaki.ikincielalissatis
Error 352779 com.ionicframework.formsph191717
Error 352789 com.dancestudio_pro.kingroberts4071
Error 352792 com.mightystore
Error 352803 passiongadgets.new.qoo10.android.qstore
Error 352819 app.bidswiss.com
Error 352822 com.meankeen
Error 352831 com.Kimetsu.AhmadDroid
Error 352832 app.scanpay
Error 352842 io.kodular.leoszsouza14.Walpaper
Error 352847 launcher2017new.launcher2017new.launcher2017new.launcher2017new.launcher2017new.launcher

Error 353852 br.com.center.agape
Error 353855 com.eosapplock.chen.beardog
Error 353860 com.robothdwallpaper.Papath
Error 353873 com.acetrot.jjcne
Error 353882 com.happysocialite
Error 353884 incipient.photoframes.newyear
Error 353889 com.trendCoolock.uniaisie
Error 353892 com.pesa.pro.app
Error 353901 com.autumnwallpaperhd.formobilephone
Error 353914 com.sigmaprocess.metrobank
Error 353917 com.sanninoforniture.ActivityIcasher.ActivityIcasher
Error 353918 japp.ppan.skin
Error 353921 com.transaction.city
Error 353924 tod.gemphis.frozenwallpapers
Error 353929 com.rusticwoodwallpaperhd.angelaappswallpaper
Error 353932 dogsinslowmotionvideo.exa.com
Error 353936 com.assamtechstartups
Error 353940 com.thejambrong.Timercircuitfornightlamp
Error 353943 wd657722996.aab
Error 353951 com.ecwid.ShopAt.madformarkers
Error 353955 com.hashcode.walloidpro
Error 353960 com.club.factory
Error 353965 ar.com.emiliano.www
Error 353972 com.HanakoKunWallpaper.Hanako_Kun_Wallpaper
Error 353990 com.VRSoft.G317


In [21]:
err_list_d3.shape

(62144, 3)

In [ ]:
## save both the err_list and df_d3 dataset files - preprocess stage 1

In [22]:
err_list_d3.to_csv("err_listSTG1_d3.csv",index=False)

In [23]:
df_d3.to_csv("preprocessSTG1_d3.csv",index=False)

In [24]:
df_d3.head()

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy
0,Vibook,com.doantiepvien.crm,Productivity,0.0,58,False,,
1,IMOCCI,com.imocci,Social,0.0,89,False,IMOCCI is the new means of communication via A...,https://www.imocci.com/wp-content/uploads/2018...
2,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,62433,False,💌<b>Neon 3d Iron Tech Theme Keyboard</b>💌\r\n\...,http://bit.ly/EmojiThemePro
3,Nepali Congress,np.com.NepaliCongress,Social,0.0,336,False,,
4,"Sudan Flag Wallpaper: Flags, Country HD Images",com.techzit.sudanflagwallpaper,Personalization,0.0,468,False,<b>Sudan</b> mobile application provides below...,https://toolsfairy.com/privacy-policy.html


In [25]:
err_list_d3.head()

,index,app_id,category
0,0.0,com.doantiepvien.crm,Productivity
1,3.0,np.com.NepaliCongress,Social
2,22.0,com.bobby.pitch,Productivity
3,32.0,com.appix.makemoney,Finance
4,38.0,com.palmer.cooltopuprecharge.reseller,Productivity


In [ ]:
## remove the app ids from the df_d3 dataset that are in err_list_d3

In [26]:
df_d3.shape

(354364, 8)

In [27]:
rem_d3 = err_list_d3['app_id'].tolist()

In [28]:
df_d3.drop(df_d3[df_d3['App_Id'].isin(rem_d3)].index, inplace=True)

In [29]:
df_d3.shape

(292220, 8)

In [30]:
df_d3.reset_index(drop=True,inplace=True)

In [ ]:
## save the df_d3 dataset file - preprocess stage 2

In [31]:
df_d3.to_csv("preprocessSTG2_d3.csv",index=False)

In [ ]:
## collect the permissions using google-play-scraper

In [36]:
!pip install google-play-scraper

In [33]:
from google_play_scraper import permissions

In [34]:
df_d3 = pd.read_csv('preprocessSTG2_d3.csv')
df_d3.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy
0,IMOCCI,com.imocci,Social,0.0,89,False,IMOCCI is the new means of communication via A...,https://www.imocci.com/wp-content/uploads/2018...
1,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,62433,False,💌<b>Neon 3d Iron Tech Theme Keyboard</b>💌\r\n\...,http://bit.ly/EmojiThemePro


In [35]:
df_d3['Wearable sensors/Activity data'] = ''
df_d3['Cellular data settings'] = ''
df_d3['Camera'] = ''
df_d3['Wi-Fi connection information'] = ''
df_d3['Other'] = ''
df_d3['Location'] = ''
df_d3['Microphone'] = ''
df_d3['Photos/Media/Files'] = ''
df_d3['Device & app history'] = ''
df_d3['Calendar'] = ''
df_d3['Contacts'] = ''
df_d3['SMS'] = ''
df_d3['Storage'] = ''
df_d3['Uncategorized'] = ''
df_d3['Device ID & call information'] = ''
df_d3['Identity'] = ''
df_d3['Phone'] = ''

In [37]:
remove_list_d3 = pd.DataFrame()

In [38]:
df_d3.head()

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Wearable sensors/Activity data,Cellular data settings,...,Photos/Media/Files,Device & app history,Calendar,Contacts,SMS,Storage,Uncategorized,Device ID & call information,Identity,Phone
0,IMOCCI,com.imocci,Social,0.0,89,False,IMOCCI is the new means of communication via A...,https://www.imocci.com/wp-content/uploads/2018...,,,...,,,,,,,,,,
1,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,62433,False,💌<b>Neon 3d Iron Tech Theme Keyboard</b>💌\r\n\...,http://bit.ly/EmojiThemePro,,,...,,,,,,,,,,
2,"Sudan Flag Wallpaper: Flags, Country HD Images",com.techzit.sudanflagwallpaper,Personalization,0.0,468,False,<b>Sudan</b> mobile application provides below...,https://toolsfairy.com/privacy-policy.html,,,...,,,,,,,,,,
3,BuradaOto,com.rzmobile.buradaoto,Shopping,4.9,4103,False,Tüm Türkiye'nin ücretsiz ilan verme platformu....,https://www.buradaoto.com/gizlilik-politikasi-15,,,...,,,,,,,,,,
4,Niagara Falls Wallpapers,com.niagara.wallpapers.backgrounds_0001,Personalization,5.0,825,False,Niagara Falls is located in the east of North ...,NaN,,,...,,,,,,,,,,


In [39]:
i = 0

In [40]:
perm_set = set()

In [41]:
for index, row in df_d3[0:292220].iterrows():
    #299763 ## 50000 100000 150000 200000 250000 286622
    app_id = row.App_Id
    #print ("Trying...",index,app_id)
    try:
        result = permissions(app_id)
        for x in list(result):
            perm_set.add(x)
            #print(x)
            if x != None:
                df_d3.at[index, x] = result[x]

    except TypeError as e:
        remove_list_d3.at[i,'index'] = index
        remove_list_d3.at[i,'app_id'] = app_id
        remove_list_d3.at[i,'category'] = row.Category
        i = i + 1
        print ("Error",index,app_id)

Error 1090 freefire.id
Error 1757 messenger.noviosymas
Error 2643 com.cta.midway
Error 6922 com.iwaiterapp.oceanpearlruthin
Error 7244 com.corinthiansfuteboltime.papeldeparedewallpaperjosari
Error 8711 com.usage.appusagemoniter
Error 8858 com.shopgate.android.app27794
Error 12070 com.WeddingInvitationCardMakerAppVision
Error 13180 com.gws.guess.prize.bond.number
Error 13714 com.swipalot.staugustine
Error 14205 com.davidani.app
Error 15020 com.iwaiterapp.thecantoneseboxdarwen
Error 15530 com.connect.dialer
Error 16154 com.gau.go.launcherex.theme.halloween.qx.threed
Error 17873 net.authoriti.authoriti
Error 21319 com.sunnytech.flowerswallpapershd
Error 21957 com.grofers.fmcdandroid
Error 23971 com.WildWaterfallsLiveWallpaperBest
Error 25458 com.mf.mfazerbaijan
Error 26160 com.inpeace.adcrescer.sp
Error 26660 li.yapp.appC0C35CE3
Error 26661 net.cityhive.store.turnpikemar
Error 26961 com.pack.myshiftwork
Error 27399 com.multimoney.multimoney.cr
Error 29321 com.iwaiterapp.bestkebabgrill
Err

In [42]:
print(perm_set)

{'Storage', 'Device & app history', 'Uncategorized', 'Microphone', 'Other', 'Phone', 'SMS', 'Cellular data settings', 'Device ID & call information', 'Wearable sensors/Activity data', 'Contacts', 'Calendar', 'Wi-Fi connection information', 'Camera', 'Identity', 'Location', 'Photos/Media/Files'}


In [43]:
remove_list_d3.tail(3)

,index,app_id,category
162,270484.0,social.toldya,Social
163,272430.0,com.nsm.user.loyaltyapp,Shopping
164,283250.0,com.happywedding100.myapp,Shopping


In [44]:
df_d3.head(5)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Wearable sensors/Activity data,Cellular data settings,...,Photos/Media/Files,Device & app history,Calendar,Contacts,SMS,Storage,Uncategorized,Device ID & call information,Identity,Phone
0,IMOCCI,com.imocci,Social,0.0,89,False,IMOCCI is the new means of communication via A...,https://www.imocci.com/wp-content/uploads/2018...,,,...,[modify or delete the contents of your USB sto...,,,[read your contacts],,[modify or delete the contents of your USB sto...,[receive data from Internet],,,
1,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,62433,False,💌<b>Neon 3d Iron Tech Theme Keyboard</b>💌\r\n\...,http://bit.ly/EmojiThemePro,,,...,[modify or delete the contents of your USB sto...,,,,,[modify or delete the contents of your USB sto...,[receive data from Internet],,,
2,"Sudan Flag Wallpaper: Flags, Country HD Images",com.techzit.sudanflagwallpaper,Personalization,0.0,468,False,<b>Sudan</b> mobile application provides below...,https://toolsfairy.com/privacy-policy.html,,,...,[modify or delete the contents of your USB sto...,,,,,[modify or delete the contents of your USB sto...,,,,
3,BuradaOto,com.rzmobile.buradaoto,Shopping,4.9,4103,False,Tüm Türkiye'nin ücretsiz ilan verme platformu....,https://www.buradaoto.com/gizlilik-politikasi-15,,,...,[modify or delete the contents of your USB sto...,,,,,[modify or delete the contents of your USB sto...,[receive data from Internet],,,
4,Niagara Falls Wallpapers,com.niagara.wallpapers.backgrounds_0001,Personalization,5.0,825,False,Niagara Falls is located in the east of North ...,NaN,,,...,[modify or delete the contents of your USB sto...,,,,,[modify or delete the contents of your USB sto...,,,,


In [45]:
remove_list_d3.shape

(165, 3)

In [46]:
df_d3.shape

(292220, 25)

In [47]:
remove_list_d3.head(5)

,index,app_id,category
0,1090.0,freefire.id,Shopping
1,1757.0,messenger.noviosymas,Social
2,2643.0,com.cta.midway,Shopping
3,6922.0,com.iwaiterapp.oceanpearlruthin,Shopping
4,7244.0,com.corinthiansfuteboltime.papeldeparedewallpa...,Personalization


In [ ]:
## save both the remove_list_d3 and df_d3 dataset files - preprocess stage 3

In [48]:
remove_list_d3.to_csv("rem_listSTG3_d3.csv",index=False)

In [49]:
df_d3.to_csv("preprocessSTG3_d3.csv",index=False)

In [ ]:
## remove the app ids from the df_d3 dataset that are in remove_list_d3

In [50]:
df_d3.shape

(292220, 25)

In [51]:
rem_d3 = remove_list_d3['app_id'].tolist()

In [52]:
df_d3.drop(df_d3[df_d3['App_Id'].isin(rem_d3)].index, inplace=True)

In [53]:
df_d3.reset_index(drop=True,inplace=True)

In [54]:
df_d3.shape

(292055, 25)

In [55]:
df_d3.head()

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Wearable sensors/Activity data,Cellular data settings,...,Photos/Media/Files,Device & app history,Calendar,Contacts,SMS,Storage,Uncategorized,Device ID & call information,Identity,Phone
0,IMOCCI,com.imocci,Social,0.0,89,False,IMOCCI is the new means of communication via A...,https://www.imocci.com/wp-content/uploads/2018...,,,...,[modify or delete the contents of your USB sto...,,,[read your contacts],,[modify or delete the contents of your USB sto...,[receive data from Internet],,,
1,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,62433,False,💌<b>Neon 3d Iron Tech Theme Keyboard</b>💌\r\n\...,http://bit.ly/EmojiThemePro,,,...,[modify or delete the contents of your USB sto...,,,,,[modify or delete the contents of your USB sto...,[receive data from Internet],,,
2,"Sudan Flag Wallpaper: Flags, Country HD Images",com.techzit.sudanflagwallpaper,Personalization,0.0,468,False,<b>Sudan</b> mobile application provides below...,https://toolsfairy.com/privacy-policy.html,,,...,[modify or delete the contents of your USB sto...,,,,,[modify or delete the contents of your USB sto...,,,,
3,BuradaOto,com.rzmobile.buradaoto,Shopping,4.9,4103,False,Tüm Türkiye'nin ücretsiz ilan verme platformu....,https://www.buradaoto.com/gizlilik-politikasi-15,,,...,[modify or delete the contents of your USB sto...,,,,,[modify or delete the contents of your USB sto...,[receive data from Internet],,,
4,Niagara Falls Wallpapers,com.niagara.wallpapers.backgrounds_0001,Personalization,5.0,825,False,Niagara Falls is located in the east of North ...,NaN,,,...,[modify or delete the contents of your USB sto...,,,,,[modify or delete the contents of your USB sto...,,,,


In [ ]:
## save the df_d3 dataset file - preprocess stage 4

In [56]:
df_d3.to_csv("preprocessSTG4_d3.csv",index=False)